In [25]:
import gurobipy as gp
import json
from gurobipy import GRB
model = gp.Model("milp")


In [26]:
def read_graph(file):
    file = open(file, "r")
    first_line = file.readline().replace("\n", "").split(" ")
    vnf_types = int(first_line[0])
    second_line = int(file.readline())
    list_servers = {}
    list_swtiches = {}
    for i in range(second_line):
        line = file.readline().replace("\n", "").split(" ")
        description = {}
        if int(line[2]) != -1:
            description["delay"] = int(line[1])
            description["active_cost"] = int(line[2])
            list_cosst = []
            for j in range(vnf_types):
                list_cosst.append(int(line[3 + j]))
            description["list_install_cost"] = list_cosst
            list_servers.update({int(line[0]): description})
        else:
            description["delay"] = int(line[1])
            list_swtiches.update({int(line[0]): description})
    number_edges = int(file.readline())
    edges = dict()
    for i in range(number_edges):
        line = file.readline().replace("\n", "").split(" ")
        edges[(int(line[0]), int(line[1]))] = int(line[2])
    list_vnfs = [i for i in range(vnf_types)]
    return list_servers, list_swtiches, edges, list_vnfs

def read_sfc(filepath:str):
    try:
        file = open(filepath, "r")

        nums_request = int(file.readline())
        list_request = []
        for i in range(nums_request):
            line = file.readline().replace("\n", "").split(" ")
            description = {}
            description['memory'] = int(line[0])
            description['bandwidth'] = int(line[1])
            description['cpu'] = int(line[2])
            description['source'] = int(line[3])
            description['destination'] = int(line[4])
            vnfs_chain = []
            for i in range (5, len(line)):
                vnfs_chain.append(int(line[i]))
            description['vnfs_chain'] = vnfs_chain
            list_request.append(description)
        file.close()
        with open("request.json", "w") as outfile:
            json.dump(list_request, outfile, indent=4)
        return list_request
    except Exception as e:
        print(e)
        return None
    


In [27]:
server_detail ,switch_detail ,edge_detail, VNFs = read_graph("input.txt") # V: servers, S: switches, E: edges, VNF: vnf types

request_detail = read_sfc("request10.txt")
R = [i for i in range(len(request_detail))]
V = list(server_detail.keys())
S = list(switch_detail.keys())
E = list(edge_detail.keys())


In [28]:
print("len(V): ", len(V))
print("len(S): ", len(S))
print("len(E): ", len(E))
print("len(VNF): ", len(VNFs))
print("len(R): ", len(R))

len(V):  10
len(S):  94
len(E):  116
len(VNF):  10
len(R):  10


In [29]:
#tính bandwidth tối đa có thể của mỗi cạnh
def max_bandwidth(request_detal):
    max_bandwith = 0
    for i in range(len(request_detal)):
        max_bandwith += request_detal[i]['bandwidth']*(len(request_detal[i]['vnfs_chain'])+1) 
    return max_bandwith


#tính memory tối đa có thể của mỗi switch_node
def max_memory(request_detal):
    max_memory = 0
    for i in range(len(request_detal)):
        max_memory += request_detal[i]['memory']*(len(request_detal[i]['vnfs_chain'])+1)
    return max_memory


#tính cpu tối đa có thể của mối server_node
def max_cpu(request_detal):
    max_cpu = 0
    for i in range(len(request_detal)):
        max_cpu += request_detal[i]['cpu']*(len(request_detal[i]['vnfs_chain']))
    return max_cpu

In [30]:
# Biến x_v: máy chủ v có hoạt động hay không
x = model.addVars(V, vtype=GRB.BINARY, name="x_v")

# Biến x_{vk}^j: VNF f_k được phục vụ trong máy chủ v cho dịch vụ j
x_j_vk = model.addVars(V, VNFs, R, vtype=GRB.BINARY, name="x_j_vk")

# Biến x'_{vk}: f_k được cài đặt trên máy chủ v cho bất kỳ dịch vụ nào
x_prime_vk = model.addVars(V, VNFs, vtype=GRB.BINARY, name="x_prime_vk")

# Biến y_{vu}^j: SFC r_j đi qua liên kết (v, u) hoặc (u, v)
y_vu_j = model.addVars(E, R, vtype=GRB.BINARY, name="y_vu_j")




In [31]:
def minimum_delay(E,V,R,VNF,request_detal,y_vu_j,x_j_vk,server_detail,edge_detail):
    edge_delay = 0
    server_delay = 0
    for index in R:
        for (u,v) in E:
            edge_delay += y_vu_j[u,v,index]*edge_detail[(u,v)]
        for v in V:
            for f in VNF:
                if f in request_detal[index]['vnfs_chain']:
                    server_delay+=x_j_vk[v,f,index] * server_detail[v]['delay']
    current_delay = edge_delay+server_delay
    max_delay = 0
    for i in edge_detail.values():
        max_delay+=i
    for s in server_detail.keys():
        max_delay+=server_detail[s]['delay']
    max_delay*=len(R)
    return current_delay/max_delay

def minimum_cost_active(V,server_detail,x):
    cost_active = 0
    for i in V:
        cost_active+=x[i]*server_detail[i]['active_cost']
    max_cost = sum(server_detail[i]['active_cost'] for i in server_detail)
    return cost_active/max_cost

def minimum_cost_install_vnf(V,VNF,x,x_prime_vk,server_detail):
    cost_install  = 0
    for i in V:
        for fk in VNF:
            cost_install+=x[i]*x_prime_vk[i,fk] * server_detail[i]['list_install_cost'][fk]
    max_cost_install = 0
    for i in server_detail.keys():
        for j in server_detail[i]['list_install_cost']:
            max_cost_install+=j
    return cost_install/max_cost_install


        
                
        

In [32]:
print("size V is: ", len(V))
print("size S is: ", len(S))
print("size E is: ", len(E))
print("size VNF is: ", len(VNF))
print("size R is: ", len(R))

size V is:  10
size S is:  94
size E is:  116
size VNF is:  10
size R is:  10


In [33]:
print(server_detail)

{40: {'delay': 325, 'active_cost': 8177, 'list_install_cost': [1345, 1412, 1020, 1125, 1076, 1801, 1368, 1514, 1848, 1219]}, 17: {'delay': 434, 'active_cost': 8539, 'list_install_cost': [1290, 1549, 1045, 1014, 1749, 1970, 1442, 1918, 1271, 1896]}, 33: {'delay': 412, 'active_cost': 5641, 'list_install_cost': [1799, 1015, 1888, 1651, 1498, 1995, 1336, 1384, 1392, 1788]}, 46: {'delay': 491, 'active_cost': 9206, 'list_install_cost': [1136, 1824, 1532, 1177, 1104, 1619, 1399, 1622, 1612, 1608]}, 65: {'delay': 371, 'active_cost': 7413, 'list_install_cost': [1435, 1912, 1623, 1401, 1731, 1678, 1636, 1997, 1536, 1217]}, 75: {'delay': 436, 'active_cost': 6161, 'list_install_cost': [1628, 1327, 1896, 1049, 1125, 1911, 1619, 1655, 1912, 1729]}, 3: {'delay': 419, 'active_cost': 5809, 'list_install_cost': [1108, 1581, 1978, 1533, 1138, 1624, 1323, 1856, 1656, 1610]}, 9: {'delay': 292, 'active_cost': 6277, 'list_install_cost': [1686, 1353, 1419, 1353, 1954, 1319, 1117, 1716, 1054, 1145]}, 15: {'del

In [34]:
DL = minimum_delay(E,V,R,VNF,request_detail,y_vu_j,x_j_vk,server_detail,edge_detail)
CS = minimum_cost_active(V,server_detail,x)
CV = minimum_cost_install_vnf(V,VNF,x,x_prime_vk,server_detail)

In [35]:
total_objective = DL+CS+CV
model.setObjective(total_objective, GRB.MINIMIZE)

In [36]:
bandwidth_limit = max_bandwidth(request_detail)
memory_limit = max_memory(request_detail)
cpu_limit = max_cpu(request_detail)

In [37]:
#thêm ràng buộc cho bandwidth
for (u,v) in E:
    model.addConstr(gp.quicksum(y_vu_j[u,v,j]*request_detail[j]['bandwidth'] for j in R) <= bandwidth_limit, name="bandwidth_limit")
    
#thêm ràng buộc cho memory
for (u,v) in E:
    if u in S:
        model.addConstr(gp.quicksum(y_vu_j[u,v,j]*request_detail[j]['memory'] for j in R) <= memory_limit, name="memory_limit")
#thêm ràng buộc cho cpu
for u in V:
    curr_cpu = 0
    for j in R:
        for k in VNF:
            curr_cpu+=x_j_vk[u,k,j]*request_detail[j]['cpu']
    model.addConstr(curr_cpu <= cpu_limit, name="cpu_limit")

#thêm ràng buộc cho số lượng VNF được cài đặt trên mỗi máy chủ

for u in V:
    num_install = 0
    for k in VNF:
            num_install+=x_prime_vk[u,k]
    model.addConstr(num_install <= len(VNF), name="num_install")
    


In [40]:
print(model.NumVars)
print(model.NumConstrs)

2270
234


In [39]:
# Reduce the size of the model by applying model simplification techniques such as constraint aggregation or variable reduction.
# Then, re-run the optimization.

model.optimize()
model.write("model.lp")
model.write("model.sol")
if model.Status == GRB.OPTIMAL:
    print("Optimal solution found")
    for v in model.getVars():
        print('%s %g' % (v.varName, v.x))

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i3-5005U CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information